<a href="https://colab.research.google.com/github/parvathymr21/ml-algo/blob/main/Multiclass_classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression,LinearRegression
from sklearn.metrics import accuracy_score,mean_squared_error
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
    

# 6. Apply multiclass techniques one vs all and all vs all on IRIS data set using a binary classification algorithm
(a) Describe the experimental settings.
(b) Compare the performance of two multiclass techniques

In [ ]:
data4 = pd.read_excel('iris.xls')
data4.head()
data4.isna().sum()


SL                7
SW                6
PL                6
PW                0
Classification    0
dtype: int64

In [ ]:
# 3 columns are having null values SL,SW & PL
# fill null values with mean
data4['SL'].mean()
data4['SL']= data4['SL'].fillna(data4['SL'].mean())
data4['SW']= data4['SW'].fillna(data4['SW'].mean())
data4['PL']= data4['PL'].fillna(data4['PL'].mean())

data4.isna().sum()
data4_c = data4.copy()
data4.columns

Index(['SL', 'SW', 'PL', 'PW', 'Classification'], dtype='object')

In [ ]:
data4 =pd.get_dummies(data4)
d1_pd = data4.iloc[:,:-2] 
d2_pd = data4.drop(['Classification_Iris-setosa','Classification_Iris-virginica'],axis=1)
d3_pd = data4.drop(['Classification_Iris-setosa','Classification_Iris-versicolor'],axis=1)


# One vs All Multiclass classification

In [ ]:
def classifier(data):
    X = data.iloc[:,:-1]
    y = data.iloc[:,-1]
    x_train,x_test,y_train,y_test = train_test_split(X,y,test_size=0.3,random_state=42)
    LR = LogisticRegression()
    model = LR.fit(x_train,y_train)
    prob = model.predict_proba(x_test)
    return prob,model

p1,model1 = classifier(d1_pd)
#p1 = pd.DataFrame(p1)
p2,model2 = classifier(d2_pd)
p3,model3 = classifier(d3_pd)
#model1.predict_proba(x_test)


X1 = data4_c.iloc[:,:-1]
y1 = data4_c.iloc[:,-1]

y1=y1.replace(to_replace=('Iris-setosa','Iris-versicolor','Iris-virginica') , value=(1,2,3))
sc= StandardScaler()
X = sc.fit_transform(X1)
X1= pd.DataFrame(X1)
x_train,x_test,y_train,y_test = train_test_split(X1,y1,test_size=0.3,random_state=42)


cls_1=(np.dot((model1.coef_),x_test.T)+model1.intercept_)
cls_2=(np.dot((model2.coef_),x_test.T)+model2.intercept_)
cls_3=(np.dot((model3.coef_),x_test.T)+model3.intercept_)

cls  =[]
for i in range(len(x_test)):
    if (cls_1[:,i]>(cls_2[:,i] and cls_3[:,i])):
        cls.append(1)
    elif (cls_2[:,i]>(cls_1[:,i] and cls_3[:,i])):
        cls.append(2)
    elif (cls_3[:,i]>(cls_1[:,i] and cls_2[:,i])):
        cls.append(3)
cls
 
def pred(y_test,cls):
    acc =0
    a=np.sum(y_test == cls)
    return a/len(y_test)

acc= pred(y_test,cls) 

print('Accuracy of the model by one vs all multiclass binary classification:',acc)

Accuracy of the model by one vs all multiclass binary classification: 0.9333333333333333


# One vs One 

In [ ]:
d1 = data4_c.copy()
d2 = data4_c.copy()
d3 = data4_c.copy()


ind1 = d1[d1['Classification']=='Iris-virginica'].index
ind1
#d1_ovo = d1.drop()
d1 = d1.drop(ind1,axis=0)
ind2 = d2[d2['Classification']=='Iris-versicolor'].index
d2 = d2.drop(ind2,axis=0)
ind3 = d3[d3['Classification']=='Iris-setosa'].index
d3 = d3.drop(ind3,axis=0)



d1_p_ovo,model1_ovo = classifier(d1)
d2_p_ovo,model2_ovo = classifier(d2)
d3_p_ovo,model3_ovo = classifier(d3)
#p1 = pd.DataFrame(p1)

X1 = data4_c.iloc[:,:-1]
y1 = data4_c.iloc[:,-1]

#y1=y1.replace(to_replace=('Iris-setosa','Iris-versicolor','Iris-virginica') , value=(1,2,3))
sc= StandardScaler()
X = sc.fit_transform(X1)
X1= pd.DataFrame(X1)
x_train,x_test,y_train,y_test = train_test_split(X1,y1,test_size=0.3,random_state=42)



d1_ovo_pred = model1_ovo.predict(x_test)
d2_ovo_pred = model2_ovo.predict(x_test)
d3_ovo_pred = model3_ovo.predict(x_test)

d_ovo_df = pd.DataFrame({'d1_ovo_pred':d1_ovo_pred,'d2_ovo_pred':d2_ovo_pred,'d3_ovo_pred':d3_ovo_pred})
d_ovo_pred = []

for i in range(len(x_test)):
    if (d1_ovo_pred[i]==d2_ovo_pred[i]==d3_ovo_pred[i]) or (d1_ovo_pred[i]==d2_ovo_pred[i]):
        d_ovo_pred.append(d1_ovo_pred[i])
    elif (d2_ovo_pred[i]==d3_ovo_pred[i]):
        d_ovo_pred.append(d2_ovo_pred[i])
    elif (d1_ovo_pred[i]==d3_ovo_pred[i]):
        d_ovo_pred.append(d1_ovo_pred[i])
    
d_ovo_df['class_pred'] = d_ovo_pred
d_ovo_df
a = pred(y_test,d_ovo_pred)
d_ovo_pred

print('Accuracy of the model by one vs one multiclass binary classification:',a)

Accuracy of the model by one vs one multiclass binary classification: 0.9777777777777777
